In [ ]:
# Cell 1: Core imports and configuration
import os
import asyncio
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple, Any
import json
import time
from dataclasses import dataclass
import numpy as np
import pandas as pd

# Core libraries
import yfinance as yf
import requests
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
import torch
import torch.nn as nn
from transformers import pipeline
import sendgrid
from sendgrid.helpers.mail import Mail
from supabase import create_client, Client
import google.generativeai as genai

# CrewAI imports
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from langchain.tools import tool

# Configuration
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Market Coverage
TICKERS = [
    "AAPL", "MSFT", "NVDA", "AMZN", "META", "GOOGL", "GOOG", "TSLA", "AVGO", "TSM",
    "COST", "ADBE", "PEP", "CSCO", "ACN", "TMUS", "CMCSA", "AMD", "INTC", "INTU",
    "AMGN", "REGN", "ADP", "QCOM", "VRTX", "AMAT", "ISRG", "MU", "ATVI", "MDLZ",
    "PYPL", "ADSK", "MELI", "KLAC", "SNPS", "CDNS", "ASML", "CHTR", "MNST", "LRCX",
    "ORLY", "KDP", "DXCM", "MAR", "IDXX", "CTAS", "ROST", "WDAY", "PCAR", "AZN",
    "CPRT", "XEL", "DLTR", "FAST", "VRSK", "ANSS", "SGEN", "BIIB", "ALGN", "SIRI",
    "EBAY", "EXC", "NTES", "JD", "BIDU", "SWKS", "INCY", "WBA", "ULTA", "TTWO",
    "VRSN", "LULU", "MTCH", "ZM", "DOCU", "OKTA", "DDOG", "CRWD", "NET", "FTNT",
    "ZS", "PANW", "TEAM", "PLTR", "DBX", "AFRM", "COIN", "HOOD", "DASH", "RIVN",
    "LCID", "PTON", "ABNB", "UBER", "LYFT", "SNOW", "DDOG", "MDB", "TWLO", "SPLK",
    "BRK.B", "JPM", "V", "MA", "BAC", "WFC", "GS", "AXP", "MS", "BLK", "C",
    "XOM", "CVX", "LLY", "JNJ", "UNH", "ABBV", "MRK", "ABT", "PFE", "MDT",
    "WMT", "PG", "KO", "HD", "MCD", "PEP", "TJX", "NKE", "SBUX", "MDLZ", "CL",
    "GE", "UNP", "CAT", "HON", "RTX", "LMT", "UPS", "NOC", "BA", "CRM"
]

# Environment variables (set these in your environment)
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
NEWS_API_KEY = os.getenv("NEWS_API_KEY")
SENDGRID_API_KEY = os.getenv("SENDGRID_API_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Initialize services
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY) if SUPABASE_URL else None
genai.configure(api_key=GEMINI_API_KEY) if GEMINI_API_KEY else None

@dataclass
class MarketEvent:
    timestamp: datetime
    event_type: str
    description: str
    affected_tickers: List[str]
    severity_score: float
    source_url: str

@dataclass
class RiskAssessment:
    event: MarketEvent
    impact_analysis: str
    probability_score: float
    time_horizon: str
    recommended_actions: List[str]